# Xor with Pytorch: Dataset and DataLoader

We now add a `Dataset` and wrap it in a `Dataloader`

**Note:** Obviously, creating a custom Dataset is not necesary for such a simple network, this is only for demo/learning pourposes

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pathlib import Path
from torch.utils.data import Dataset, DataLoader

Create dataset

In [ ]:
class DatasetXor(Dataset):
    """ 
    Custom dataset: We have to override methods __len__ and __getitem__
    """
    def __init__(self, num):
        x1 = np.random.randn(num)
        x2 = np.random.randn(num)
        x = np.stack([x1, x2], axis=1).astype('float32')
        y = (2.0 * np.logical_xor(x1 >= 0, x2 >= 0).astype('float32') - 1.0) * 0.9
        self.x, self.y = torch.Tensor(x), torch.Tensor(y)
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

Create model

In [ ]:
# Module class example. It does exactly the same as:
#     nn.Sequential(nn.Linear(...), nn.Tanh(), nn.Linear(...), nn.Tanh())
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(2,3)
        self.layer_2 = nn.Linear(3,1)

    def forward(self, x):
        x = self.layer_1(x)
        x = torch.tanh(x)
        x = self.layer_2(x)
        out = torch.tanh(x)
        return out

Train model: Now we can use Dataloader's batch

In [ ]:
def train(model, dataloader, epochs, lr, momentum):
    device = torch.device("cpu")
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    for n_epoch in range(epochs):
        for n_batch, batch in enumerate(dataloader):
            x, y = batch
            optimizer.zero_grad()
            output = model(x)
            loss = F.mse_loss(output.squeeze(), y)
            loss.backward()
            optimizer.step()
            if n_epoch % 50 == 0 and n_batch == 0:
                print(f"Train Epoch: {n_epoch} / {epochs}\tn_batch: {n_batch}\tLoss: {loss.item():.6f}")

### Run

In [ ]:
dataset_xor = DatasetXor(1000)
dataloader = DataLoader(dataset_xor, batch_size=100)
model = Net()
train(model, dataloader, epochs=1000, lr=0.05, momentum=0.8)

### Show results

In [ ]:
x_train, y_train = dataset_xor.x, dataset_xor.y
yhat = model(x_train)
for i in range(len(yhat)):
    print(f"{y_train[i]}\t{yhat[i]}\t{x_train[i]}")